<table style="width: 100%; border-collapse: collapse;" border="0">
<tr>
<td><b>Created:</b> Monday 30 January 2017</td>
<td style="text-align: right;"><a href="https://www.github.com/rhyswhitley/fire_limitation">github.com/rhyswhitley/fire_limitation</td>
</tr>
</table>

<div>
<center>
<font face="Times">
<br>
<h1>Quantifying the uncertainity of a global fire limitation model using Bayesian inference</h1>
<h2>Part 1: Staging data for analysis</h2>
<br>
<br>
<sup>1,* </sup>Douglas Kelley, 
<sup>2 </sup>Ioannis Bistinas, 
<sup>3, 4 </sup>Chantelle Burton, 
<sup>1 </sup>Tobias Marthews, 
<sup>5 </sup>Rhys Whitley
<br>
<br>
<br>
<sup>1 </sup>Centre for Ecology and Hydrology, Maclean Building, Crowmarsh Gifford, Wallingford, Oxfordshire, United Kingdom
<br>
<sup>2 </sup>Vrije Universiteit Amsterdam, Faculty of Earth and Life Sciences, Amsterdam, Netherlands
<br>
<sup>3 </sup>Met Office United Kingdom, Exeter, United Kingdom
<br>
<sup>4 </sup>Geography, University of Exeter, Exeter, United Kingdom
<br>
<sup>5 </sup>Natural Perils Pricing, Commercial & Consumer Portfolio & Pricing, Suncorp Group, Sydney, Australia
<br>
<br>
<h3>Summary</h3>
<hr>
<p> 
This notebook aims to process the separate netCDF4 files for the model drivers (X<sub>i=1, 2, ... M</sub>) and model target (Y) into a unified tabular data frame, exported as a compressed comma separated value (CSV) file. This file is subsequently used in the Bayesian inference study that forms the second notebook in this experiment. The advantage of the pre-processing the data separately to the analysis allows for it be quickly staged on demand. Of course other file formats may be more advantageous for greater compression (e.g. SQLite3 database file).
</p>
<br>
<b>You will need to run this notebook to prepare the dataest before you attempt the Bayesian analysis in Part 2</b>.
<br>
<br>
<br>
<i>Python code and calculations below</i>
<br>
<hr>
</font>
</center>
</div>

## Load libraries

Changed

In [33]:
# data munging and analytical libraries 
import re
import os
import numpy as np
import pandas as pd
from netCDF4 import Dataset 

# graphical libraries
import matplotlib.pyplot as plt
%matplotlib inline

# set paths
inPath = "../outputs/amazon_region/"
outPath = "../data/amazon_inference_data"

satalites = ["TERRA_M__M", "TERRA_M__T", "AQUA_M__M", "AQUA_M__T"]

nmonths_import = 16*12

## Import and clean data

Set the directory path and look for all netcdf files that correspond to the model drivers and target.

In [34]:
driver_paths_all = [os.path.join(dp, f) for (dp, _, fn) in os.walk(inPath) for f in fn if f.endswith('.nc')]
driver_paths_all = [path for path in driver_paths_all if path != '../outputs/amazon_region/burnt_area-GFED4s_2.5degree_2001-2016.nc']
driver_names_all = [re.search('^[a-zA-Z_]*', os.path.basename(fp)).group(0) for fp in driver_paths_all]

file_table = pd.DataFrame({'filepath': driver_paths_all, 'file_name': driver_names_all})

def checkFilename(driver_name, sat):
    
    if "firecount" in driver_name:
        if sat in driver_name: return True
        return False
    
    return True

def select_files_for_sat(sat):
    test = [checkFilename(driver_name, sat) for driver_name in driver_names_all]
    
    driver_paths = np.array(driver_paths_all)[test]
    driver_names = np.array(driver_names_all)[test]
    
    driver_names = ["firecount" if "firecount" in driver_name else driver_name for driver_name in driver_names]
    return driver_paths, driver_names
    
driver_info = [select_files_for_sat(sat) for sat in satalites]

Define a function to extract the variable values from each netCDF4 file. Variables are flattened from a 3 dimensional array to 1 dimensional version, pooling all values both spatially and temporily. 

Don't know if this is the correct way to do this, but will come back to it once I understand the model (and its optimisation) better.

In [35]:
from pdb import set_trace as browser

def nc_extract(fpath):
    print("Processing: {0}".format(fpath))
    with Dataset(fpath, 'r') as nc_file:
        gdata = np.array(nc_file.variables['variable'][:,:,:])
        if (len(gdata) < nmonths_import): return None
        gdata = gdata[13:nmonths_import]
        gdata[gdata < -9E9] = np.nan
        gflat = gdata.flatten()
        if type(gdata) == np.ma.core.MaskedArray:
            return gflat[~gflat.mask].data
        else:
            return gflat.data

Execute the above function on all netCDF4 file paths.

In [36]:
values = [ [nc_extract(dp) for dp in driver_paths[0]] for driver_paths in driver_info]

Processing: ../outputs/amazon_region/vegetation/nonetreecover-2001-June2018.nc
Processing: ../outputs/amazon_region/vegetation/treecover-2001-June2018.nc
Processing: ../outputs/amazon_region/vegetation/vegcover-2001-June2018.nc
Processing: ../outputs/amazon_region/vegetation/MaxOverMean_soilw.0-10cm.gauss.2001-2019.nc
Processing: ../outputs/amazon_region/human/cropland2001-2019.nc
Processing: ../outputs/amazon_region/human/fract_agr2001-2019.nc
Processing: ../outputs/amazon_region/human/pasture2001-2019.nc
Processing: ../outputs/amazon_region/human/population_density-PD_HYDEv3.2_2001-2018.nc
Processing: ../outputs/amazon_region/human/dt_cropland2001-2019.nc
Processing: ../outputs/amazon_region/human/dt_fract_agr2001-2019.nc
Processing: ../outputs/amazon_region/human/dt_pasture2001-2019.nc
Processing: ../outputs/amazon_region/human/dt-cropland2001-2019.nc
Processing: ../outputs/amazon_region/human/dt-fract_agr2001-2019.nc
Processing: ../outputs/amazon_region/human/dt-pasture2001-2019.nc

Turn this into a dataframe for the analysis.

In [37]:
# turn list into a dataframe
def makeDataFrave(value, driver_names):
    df = pd.DataFrame(np.array(value).T, columns=driver_names)
    print(df.info())
    df.dropna(inplace=True)
    return df
fire_df = [makeDataFrave(value, driver_names[1]) for value, driver_names in zip(values, driver_info)]
#fire_df.info()
# replace null flags with pandas null
#fire_df.replace(fire_df < -3e38, np.nan, inplace=True)
#fire_df[] = np.nan
# drop all null rows (are ocean and not needed in optim)
#fire_df.dropna(inplace=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 114202 entries, 0 to 114201
Data columns (total 23 columns):
nonetreecover         44750 non-null float32
treecover             44750 non-null float32
vegcover              44750 non-null float32
MaxOverMean_soilw     44750 non-null float32
cropland              44750 non-null float32
fract_agr             44750 non-null float32
pasture               44750 non-null float32
population_density    44750 non-null float32
dt_cropland           44750 non-null float32
dt_fract_agr          44750 non-null float32
dt_pasture            44750 non-null float32
dt                    44750 non-null float32
dt                    44750 non-null float32
dt                    44750 non-null float32
air                   44750 non-null float32
emc                   44750 non-null float32
lightning             44750 non-null float32
relative_humidity     44750 non-null float32
tasMax_               44750 non-null float32
precip                44750 non-nu

Check that we've built it correctly.

In [38]:
for df in fire_df:
    print(df.head(10))

    nonetreecover  treecover  vegcover  MaxOverMean_soilw  cropland  \
25       0.566104   0.288708  0.854813           1.279859  0.075360   
26       0.519627   0.299700  0.819326           1.502096  0.084143   
27       0.554406   0.234010  0.788417           1.783515  0.097023   
28       0.559727   0.243175  0.802903           2.143794  0.062484   
29       0.562847   0.158067  0.720914           1.937969  0.044306   
46       0.476833   0.408024  0.884858           1.062662  0.017306   
47       0.511510   0.403021  0.914531           1.062974  0.049049   
48       0.549206   0.347227  0.896432           1.193181  0.057812   
49       0.597786   0.269141  0.866927           1.383104  0.066443   
50       0.591602   0.240000  0.831602           1.698555  0.041235   

    fract_agr   pasture  population_density  dt_cropland  dt_fract_agr  ...  \
25   0.586741  0.511381           55.566036     0.000075      0.005090  ...   
26   0.447438  0.363295           62.798744     0.000132    

Export this to disk to be used by the analysis notebook - used gzip compression to save on space. Beware, because of there are approximation 10 million rows of data, this may take some time.

In [39]:
for sat, df in zip(satalites, fire_df):
    outPathi = outPath + '-' + sat + '.csv'
    savepath = os.path.expanduser(outPathi)
    df.to_csv(savepath, index=False)

<div>
<br>
<br>
<br>
<center>
<font size="5">
<a style="font-weight: bold; size: 5" href="http://localhost:8888/notebooks/notebooks/bayesian_inference.ipynb">Part 2: click here</a>
</font>
</center>
</div>